In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install pyspark 
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 281.4 MB 32 kB/s 
     |████████████████████████████████| 198 kB 53.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=845b055afb593ecb42b90f523a730e60f5c45a27908fa3aa7126943b8aff334a
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 39 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Rea

In [6]:
cur_path = "/content/drive/MyDrive/BigData/finalProject/"
os.chdir(cur_path)

In [247]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('final').getOrCreate()

In [248]:
data = spark.read.csv(cur_path + 'car_prices.csv',inferSchema=True,header=True)

In [249]:
data.show()

+----+---------+-------------------+--------------------+-----------+------------+-----------------+-----+---------+--------+------+--------+--------------------+-----+------------+--------------------+
|year|     make|              model|                trim|       body|transmission|              vin|state|condition|odometer| color|interior|              seller|  mmr|sellingprice|            saledate|
+----+---------+-------------------+--------------------+-----------+------------+-----------------+-----+---------+--------+------+--------+--------------------+-----+------------+--------------------+
|2015|      Kia|            Sorento|                  LX|        SUV|   automatic|5xyktca69fg566472|   ca|        5| 16639.0| white|   black|kia motors americ...|20500|       21500|Tue Dec 16 2014 1...|
|2015|      Kia|            Sorento|                  LX|        SUV|   automatic|5xyktca69fg561319|   ca|        5|  9393.0| white|   beige|kia motors americ...|20800|       21500|Tue Dec

In [250]:
data.summary().show()

+-------+------------------+------+-----------------+-----------------+------------+------------+-----------------+-----------------+------------------+-----------------+-----------------+--------+--------------------+--------------------+------------------+--------------------+
|summary|              year|  make|            model|             trim|        body|transmission|              vin|            state|         condition|         odometer|            color|interior|              seller|                 mmr|      sellingprice|            saledate|
+-------+------------------+------+-----------------+-----------------+------------+------------+-----------------+-----------------+------------------+-----------------+-----------------+--------+--------------------+--------------------+------------------+--------------------+
|  count|            558837|548536|           548438|           548186|      545642|      493484|           558833|           558837|            547043|        

## Missing data

In [251]:
from pyspark.sql.functions import isnan, when, count, col
data.select([count(when(isnan('year'),True))]).show()

+------------------------------------------+
|count(CASE WHEN isnan(year) THEN true END)|
+------------------------------------------+
|                                         0|
+------------------------------------------+



In [252]:
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]).show()

+----+-----+-----+-----+-----+------------+---+-----+---------+--------+-----+--------+------+---+------------+--------+
|year| make|model| trim| body|transmission|vin|state|condition|odometer|color|interior|seller|mmr|sellingprice|saledate|
+----+-----+-----+-----+-----+------------+---+-----+---------+--------+-----+--------+------+---+------------+--------+
|   0|10301|10399|10651|13195|       65353|  4|    0|    11794|      94|  749|     749|     0|  0|           0|       0|
+----+-----+-----+-----+-----+------------+---+-----+---------+--------+-----+--------+------+---+------------+--------+



In [253]:
data.count()

558837

In [254]:
from pyspark.sql.functions import *
amount_missing_df = data.select([(count(when(isnan(c) | col(c).isNull(), c))/count(lit(1)) * 100).alias(c) for c in data.columns])
amount_missing_df.show()

+----+------------------+-----------------+------------------+-----------------+------------------+-------------------+-----+-----------------+--------------------+-------------------+-------------------+------+---+------------+--------+
|year|              make|            model|              trim|             body|      transmission|                vin|state|        condition|            odometer|              color|           interior|seller|mmr|sellingprice|saledate|
+----+------------------+-----------------+------------------+-----------------+------------------+-------------------+-----+-----------------+--------------------+-------------------+-------------------+------+---+------------+--------+
| 0.0|1.8432924090566658|1.860828828441925|1.9059224782897337|2.361153610086662|11.694465470253402|7.15772219806491E-4|  0.0|2.110454390099439|0.016820647165452538|0.13402834815876544|0.13402834815876544|   0.0|0.0|         0.0|     0.0|
+----+------------------+-----------------+-----

# 1. Impute Missing Data

## 1) Transmission

Transmission has the largest percent of missing value, which is almost 12%. Other things are less than 3%.

In [255]:
data.select('transmission').distinct().show()

+------------+
|transmission|
+------------+
|        null|
|   automatic|
|      manual|
|       Sedan|
|       sedan|
+------------+



In [256]:
# Change 'Sedan' to 'sedan'
data = data.withColumn('transmission', regexp_replace('transmission', 'Sedan', 'sedan'))
data.select('transmission').distinct().show()

+------------+
|transmission|
+------------+
|        null|
|   automatic|
|      manual|
|       sedan|
+------------+



In [257]:
data.groupBy('transmission').count().show()

+------------+------+
|transmission| count|
+------------+------+
|        null| 65353|
|   automatic|475914|
|      manual| 17544|
|       sedan|    26|
+------------+------+



In [258]:
475914 / data.count()
# 85% of transmission is "automatic"
# Since majority of cars have an automatic transmission, null data will be filled with "automatic"

data = data.na.fill(value='automatic',subset=["transmission"])

## 2) body

In [259]:
data.select('body').distinct().show(88)

+--------------------+
|                body|
+--------------------+
|       G Convertible|
|                Koup|
|            Quad Cab|
|             G Sedan|
|          Access Cab|
|        Extended Cab|
|           Hatchback|
|            Club Cab|
|             Ram Van|
|             G Coupe|
|           Q60 Coupe|
|         Convertible|
|         CrewMax Cab|
|                null|
|            King Cab|
|        E-Series Van|
|            Crew Cab|
|          Double Cab|
|          Cab Plus 4|
|         Regular Cab|
|     TSX Sport Wagon|
|            SuperCab|
|                 SUV|
|         CTS-V Coupe|
|               Sedan|
|             Xtracab|
|         convertible|
|           CTS Wagon|
|           SuperCrew|
|       Genesis Coupe|
|               Wagon|
|       Elantra Coupe|
|            Cab Plus|
|                 Van|
|               sedan|
|           CTS Coupe|
|GranTurismo Conve...|
|         CTS-V Wagon|
|  Beetle Convertible|
|            Mega Cab|
|     G37 C

In [260]:
# Since the body is mixed with capital and lower letter, convert them into lower letter.
from pyspark.sql.functions import lower, col
data = data.withColumn('body', lower(col("body")))

In [261]:
data = data.withColumn('body', 
   when(data.body.contains('sedan'),'sedan') 
  .when(data.body.contains('suv'),'suv') 
  .when(data.body.contains('cab'),'cab')
  .when(data.body.contains('oup'),'coupe')
  .when(data.body.contains('van'),'van')
  .when(data.body.contains('convertible'),'convertible')
  .when(data.body.contains('wagon'),'wagon')
  .otherwise(data.body))

In [262]:
data.groupBy('body').count().show(100)

+------------+------+
|        body| count|
+------------+------+
|         van| 31959|
|       wagon| 16180|
|        null| 13195|
|       coupe| 20163|
|         suv|143844|
| convertible| 10933|
|       sedan|248760|
|   hatchback| 26237|
|   supercrew|  9033|
|         cab| 38507|
| navitgation|    26|
+------------+------+



In [263]:
data = data.na.drop(subset=["body"])

In [264]:
data.groupBy('body').count().show(100)

+------------+------+
|        body| count|
+------------+------+
|         van| 31959|
|       wagon| 16180|
|       coupe| 20163|
|         suv|143844|
| convertible| 10933|
|       sedan|248760|
|   hatchback| 26237|
|   supercrew|  9033|
|         cab| 38507|
| navitgation|    26|
+------------+------+



## 3) Condition

In [265]:
data.show()

+----+---------+-------------------+--------------------+-----------+------------+-----------------+-----+---------+--------+------+--------+--------------------+-----+------------+--------------------+
|year|     make|              model|                trim|       body|transmission|              vin|state|condition|odometer| color|interior|              seller|  mmr|sellingprice|            saledate|
+----+---------+-------------------+--------------------+-----------+------------+-----------------+-----+---------+--------+------+--------+--------------------+-----+------------+--------------------+
|2015|      Kia|            Sorento|                  LX|        suv|   automatic|5xyktca69fg566472|   ca|        5| 16639.0| white|   black|kia motors americ...|20500|       21500|Tue Dec 16 2014 1...|
|2015|      Kia|            Sorento|                  LX|        suv|   automatic|5xyktca69fg561319|   ca|        5|  9393.0| white|   beige|kia motors americ...|20800|       21500|Tue Dec

In [266]:
avg_con = data.agg({"condition": "avg"}).collect()[0][0]
data = data.fillna(value = str(avg_con), subset = ['condition'])

## 4) Trim

In [267]:
data.select('trim').distinct().count()

1530

In [268]:
data.groupBy('trim').count().show(1530)

+--------------------+-----+
|                trim|count|
+--------------------+-----+
|                  LT|16915|
|         G37 Journey| 2902|
|     C230 Kompressor|  111|
|             Turbo S|   27|
|    Laramie Longhorn|   10|
|         2.2 Premium|    7|
|            S Hybrid|   32|
|            LS Fleet| 2361|
|                 VR6|   12|
|            Aveo5 LS|   97|
|                 RTX|   14|
|2.0T Premium Plus...|  308|
|                PZEV|  100|
|             S65 AMG|   12|
| Special Edition V-6|   40|
|  C320 Luxury 4MATIC|    6|
|              Z71 LS|   16|
|              4.6 SE|    2|
|               Cinco|   13|
|2.5L Red Rock Edi...|   17|
|      CE 7-Passenger|  194|
|              S Plus|   45|
|            R320 CDI|   12|
|           Econoline|    7|
|                328d|   21|
|   Triple White PZEV|    4|
|         Ralliart V6|    3|
|   xDrive35i Premium|  328|
|                1500|  791|
|          C230 Sport|  147|
|              SLK300|   18|
|          Mai

In [269]:
data = data.na.drop(subset=["trim"])

## 4) Vin

In [270]:
data = data.na.drop(subset=["vin"])

## 5) make

In [271]:
data.groupBy('make').count().show(1530)

+-------------+-----+
|         make|count|
+-------------+-----+
|   Volkswagen|12577|
|   Oldsmobile|  364|
|     Infiniti|15305|
|        Lexus|11861|
|       Jaguar| 1420|
|       Saturn| 2841|
|         FIAT|  865|
|     Maserati|  133|
|        Scion| 1687|
|  Rolls-Royce|   17|
|         Jeep|15372|
|   Mitsubishi| 4140|
|          Kia|18077|
|    Chevrolet|60179|
|        Volvo| 3788|
|      Hyundai|21816|
|         Saab|  484|
|        Honda|27203|
|         MINI| 3224|
|         Audi| 5869|
|          Ram| 4574|
|     Cadillac| 7519|
|        Isuzu|  204|
|     Plymouth|   20|
|      Pontiac| 4497|
|          Geo|   19|
|   Land Rover| 1735|
|Mercedes-Benz|17141|
|      Mercury| 1992|
|       Daewoo|    3|
|      Lincoln| 5757|
|     Chrysler|17276|
|        Tesla|   23|
|          BMW|20719|
|       Suzuki| 1073|
|        Acura| 5901|
|       HUMMER|  805|
|      Ferrari|   19|
|          GMC|10613|
|        Buick| 5107|
|      Porsche| 1383|
|       Fisker|    9|
|        s

In [272]:
data = data.na.fill('Ford',subset=["make"])

## 6) color

In [273]:
data.groupBy('color').count().show(1530)

+---------+------+
|    color| count|
+---------+------+
|        —| 24451|
| charcoal|   472|
|turquoise|   224|
| burgundy|  8744|
|   orange|  1986|
|    green| 10796|
|   yellow|  1232|
|     null|   728|
|   silver| 81467|
|   purple|  1507|
|    white|103639|
|     gray| 81525|
|     pink|    42|
|     lime|    15|
|      red| 42525|
|     gold| 10898|
|off-white|  1415|
|    beige|  8905|
|    black|108660|
|    brown|  6611|
|     blue| 49774|
|    14872|     1|
|     4802|     1|
|    12655|     1|
|     1167|     1|
|     9410|     1|
|     2172|     1|
|    15719|     1|
|     6388|     1|
|     2846|     1|
|    18384|     1|
|     6864|     1|
|     5001|     1|
|     9887|     1|
|     9837|     1|
|    18561|     1|
|    11034|     1|
|    20627|     1|
|     6158|     1|
|    16633|     1|
|      339|     1|
|      721|     1|
|     5705|     1|
+---------+------+



In [274]:
data = data.filter((data.color != '14872') & (data.color != '4802') & (data.color != '12655')& (data.color != '1167')& (data.color != '1167')& (data.color != '9410')& (data.color != '2172')& (data.color != '15719')& (data.color != '6388')& (data.color != '2846')& (data.color != '18384')& (data.color != '6864')& (data.color != '5001')& (data.color != '9887')& (data.color != '9837')& (data.color != '18561')& (data.color != '11034')& (data.color != '20627')& (data.color != '6158')& (data.color != '16633')& (data.color != '339')& (data.color != '721')& (data.color != '5705'))

In [275]:
data.groupBy('color').count().show(1530)

+---------+------+
|    color| count|
+---------+------+
|        —| 24451|
| charcoal|   472|
|turquoise|   224|
| burgundy|  8744|
|   orange|  1986|
|    green| 10796|
|   yellow|  1232|
|   silver| 81467|
|   purple|  1507|
|    white|103639|
|     gray| 81525|
|     pink|    42|
|     lime|    15|
|      red| 42525|
|     gold| 10898|
|off-white|  1415|
|    beige|  8905|
|    black|108660|
|    brown|  6611|
|     blue| 49774|
+---------+------+



In [276]:
data = data.withColumn('color', 
    when(data.color.contains('charcoal'),data.color)
   .when(data.color.contains('turquoise'),data.color)
   .when(data.color.contains('burgundy'),data.color)
   .when(data.color.contains('orange'),data.color)
   .when(data.color.contains('green'),data.color)
   .when(data.color.contains('yellow'),data.color)
   .when(data.color.contains('silver'),data.color)
   .when(data.color.contains('purple'),data.color)
   .when(data.color.contains('white'),data.color)
   .when(data.color.contains('gray'),data.color)
   .when(data.color.contains('pink'),data.color)
   .when(data.color.contains('lime'),data.color)
   .when(data.color.contains('red'),data.color)
   .when(data.color.contains('gold'),data.color)
   .when(data.color.contains('off-white'),data.color)
   .when(data.color.contains('beige'),data.color)
   .when(data.color.contains('black'),data.color)
   .when(data.color.contains('brown'),data.color)
   .when(data.color.contains('blue'),data.color)
   .otherwise('black'))

In [277]:
data.groupBy('color').count().show(1530)

+---------+------+
|    color| count|
+---------+------+
| charcoal|   472|
|turquoise|   224|
| burgundy|  8744|
|   orange|  1986|
|    green| 10796|
|   yellow|  1232|
|   silver| 81467|
|   purple|  1507|
|    white|103639|
|     gray| 81525|
|     pink|    42|
|     lime|    15|
|      red| 42525|
|     gold| 10898|
|off-white|  1415|
|    beige|  8905|
|    black|133111|
|    brown|  6611|
|     blue| 49774|
+---------+------+



## 7) interior

In [278]:
data.groupBy('interior').count().show(1530)

+---------+------+
| interior| count|
+---------+------+
|        —| 16409|
| burgundy|   184|
|   orange|   135|
|    green|   225|
|   yellow|    20|
|   silver|  1065|
|   purple|   331|
|    white|   244|
|     gray|173443|
|      red|  1312|
|off-white|   480|
|     gold|   314|
|    beige| 58268|
|      tan| 42798|
|    black|240210|
|    brown|  8410|
|     blue|  1040|
+---------+------+



In [289]:
data = data.withColumn('interior', 
    when(data.interior.contains('burgundy'),data.interior)
   .when(data.interior.contains('orange'),data.interior)
   .when(data.interior.contains('green'),data.interior)
   .when(data.interior.contains('yellow'),data.interior)
   .when(data.interior.contains('silver'),data.interior)
   .when(data.interior.contains('purple'),data.interior)
   .when(data.interior.contains('white'),data.interior)
   .when(data.interior.contains('gray'),data.interior)
   .when(data.interior.contains('red'),data.interior)
   .when(data.interior.contains('off-white'),data.interior)
   .when(data.interior.contains('gold'),data.interior)
   .when(data.interior.contains('beige'),data.interior)
   .when(data.interior.contains('tan'),data.interior)
   .when(data.interior.contains('black'),data.interior)
   .when(data.interior.contains('brown'),data.interior)
   .when(data.interior.contains('blue'),data.interior)
   .otherwise('black'))

In [290]:
data.groupBy('interior').count().show(1530)

+---------+------+
| interior| count|
+---------+------+
| burgundy|   184|
|   orange|   135|
|    green|   225|
|   yellow|    20|
|   silver|  1065|
|   purple|   331|
|    white|   244|
|     gray|173443|
|      red|  1312|
|off-white|   480|
|     gold|   314|
|    beige| 58268|
|      tan| 42798|
|    black|256619|
|    brown|  8410|
|     blue|  1040|
+---------+------+



## 8) Odometer

In [291]:
avg_odo = data.agg({"odometer": "avg"}).collect()[0][0]
data = data.na.fill(value=avg_odo,subset=["odometer"])

## 9) Model

In [292]:
test = data.groupBy('model').count()

In [293]:
test.orderBy(col("count").desc()).show(truncate=False)

+--------------+-----+
|model         |count|
+--------------+-----+
|Altima        |19426|
|F-150         |14460|
|Fusion        |12904|
|Camry         |12540|
|Escape        |11834|
|Focus         |10368|
|Accord        |9124 |
|3 Series      |8186 |
|Grand Caravan |7937 |
|Impala        |7919 |
|Explorer      |7693 |
|Civic         |7430 |
|G Sedan       |7412 |
|Corolla       |7352 |
|Malibu        |7027 |
|Sonata        |6911 |
|Maxima        |6602 |
|Silverado 1500|6358 |
|Cruze         |6347 |
|Elantra       |6256 |
+--------------+-----+
only showing top 20 rows



In [294]:
data = data.na.fill(value="Altima",subset=["model"])

In [295]:
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]).show()

+----+----+-----+----+----+------------+---+-----+---------+--------+-----+--------+------+---+------------+--------+----------+
|year|make|model|trim|body|transmission|vin|state|condition|odometer|color|interior|seller|mmr|sellingprice|saledate|madeRegion|
+----+----+-----+----+----+------------+---+-----+---------+--------+-----+--------+------+---+------------+--------+----------+
|   0|   0|    0|   0|   0|           0|  0|    0|        0|       0|    0|       0|     0|  0|           0|       0|         0|
+----+----+-----+----+----+------------+---+-----+---------+--------+-----+--------+------+---+------------+--------+----------+



# 2. Feature Engineering

## 1) Vin (Vehicle identification number)
The first 2 letters of Vin means country or region codes. Except this, other information decreases the accuracy of model.

In [313]:
data = data.withColumn('madeRegion', data.vin.substr(1,2))
data = data.drop('vin')

In [314]:
data.show()

+----+---------+-------------------+--------------------+-----------+------------+-----+------------------+--------+------+--------+--------------------+-----+------------+--------+----------+
|year|     make|              model|                trim|       body|transmission|state|         condition|odometer| color|interior|              seller|  mmr|sellingprice|saledate|madeRegion|
+----+---------+-------------------+--------------------+-----------+------------+-----+------------------+--------+------+--------+--------------------+-----+------------+--------+----------+
|2015|      Kia|            Sorento|                  LX|        suv|   automatic|   ca|                 5| 16639.0| white|   black|kia motors americ...|20500|       21500|    2014|        5x|
|2015|      Kia|            Sorento|                  LX|        suv|   automatic|   ca|                 5|  9393.0| white|   beige|kia motors americ...|20800|       21500|    2014|        5x|
|2014|      BMW|           3 Series

## 2) Saledate

Saledate consists of day, date, time. However, in car market, only year does matter. For this reason, I am going to remove extra information and leave only year date.

In [15]:
data.collect()[40][15]

'Wed Feb 04 2015 04:30:00 GMT-0800 (PST)'

In [311]:
data = data.withColumn('saledate', data.saledate.substr(12,4))

In [312]:
data.show()

+----+---------+-------------------+--------------------+-----------+------------+-----------------+-----+------------------+--------+------+--------+--------------------+-----+------------+--------+----------+
|year|     make|              model|                trim|       body|transmission|              vin|state|         condition|odometer| color|interior|              seller|  mmr|sellingprice|saledate|madeRegion|
+----+---------+-------------------+--------------------+-----------+------------+-----------------+-----+------------------+--------+------+--------+--------------------+-----+------------+--------+----------+
|2015|      Kia|            Sorento|                  LX|        suv|   automatic|5xyktca69fg566472|   ca|                 5| 16639.0| white|   black|kia motors americ...|20500|       21500|    2014|        5x|
|2015|      Kia|            Sorento|                  LX|        suv|   automatic|5xyktca69fg561319|   ca|                 5|  9393.0| white|   beige|kia mo

In [315]:
data.toPandas().to_csv('cleaned_data.csv')